In [ ]:
from pyswip.prolog import Prolog
from pyswip import registerForeign, Functor, call, Variable
import tempfile
import os
import sys

In [10]:
prolog = Prolog()
prolog.consult("./kb.pl") # note: there is a commented version of the KB in kb_commented.pl (it was causing errors after comments & i couldnt find the problem)
### THIS CODE WAS ADJUSTED FROM THE EXPERT SYSTEM CLASS FOR THE LBA
### I ADJUSTED THE LBA CODE but nonetheless, it was still a collaboration with Finn, Dilnas, and Helen

# Use Prolog's functors in Python
retractall = Functor("retractall")
known = Functor("known", 3)
member = Functor('member', 2)

# user input and print functions
def read_py(A,V,Y):
    """
    Asks the user whether a specified value V of an attribute A is true,
    and binds the response to a Prolog variable.

        Inputs
    --------
    A: the specified attribute
    V: the associated value
    Y: a variable with which the user response is unified

    Output
    ------
    success/failure: returns true as long as Y is a variable, and false otherwise; maintains Prolog execution flow

    """
    if isinstance(Y, Variable):
        response = input(str(A))

        # handle variants on "Yes", allowing for whitespace and capitalisation
        processed_response = response.lower().strip()
        print(f"Question: {str(A)}, Response: {processed_response}")
        Y.unify(processed_response)
        return True
    else:
        return False

def read_menu_py(A, X, Menu):
    """
    Asks the user to specify which value V of several possible menu options in Menu
    for a given attribute A they prefer. Unifies the result with X, which is a variable
    that is then passed into Prolog's predicate logic system.

    Inputs
    -----
    A: the specified attribute
    X: the variable used to encode user responses
    Menu: the list of possible values for the attribute

    """
    if isinstance(X, Variable):
        menu_dict = {str(i+1): str(Menu[i]) for i in range(len(Menu))}
        options = '\n'.join([f'{i}. {j}' for i, j in menu_dict.items()])
        response = input(f"{str(A)}: {options}")

        if str(response) in [str(i+1) for i in range(len(Menu))]:
            print(f"Question: {str(A)}, Response: {menu_dict.get(response)} [{response}]")
        
        # fail gracefully on incorrect input and try again
        X.unify(menu_dict.get(response, str(response)))
        
        return True
    else:
        return False
        
def ask_menu_again_py(Z):
    """
    Printing function for cases where Prolog does not recognise the input to menuask
    as a legal value

    Inputs
    --------
    Z: the user input

    Outputs
    --------
    Returns true so that Prolog continues execution

    """
    print(f"{Z} is not a legal value, try again.")
    return True

# registering functions for use in prolog
read_py.arity=3
ask_menu_again_py.arity=1
read_menu_py.arity=3

registerForeign(read_py)
registerForeign(read_menu_py)
registerForeign(ask_menu_again_py)

call(retractall(known))
output = [s for s in prolog.query("synthesis(Start, 'mTOR', Input_Direction, List, Loops).")]
inputs = list([output[0]['Start'], output[0]['Input_Direction']])
paths = output[0]['List']
loops = output[0]['Loops']


if not paths:
    print(f"No documented pathways between {inputs[0]} and mTOR in this database.")
elif len(paths) == 1:
    print(f"\n Pathways from {inputs[0]} to mTOR, when {inputs[0]} is {inputs[1]}ed: \n- {paths[0][1]} : {paths[0][0]}")
else: 
    print(f"\n Pathways from {inputs[0]} to mTOR, when {inputs[0]} is {inputs[1]}ed: \n- " + '\n- '.join(map(str, ([f"{i[1]}: {i[0]}" for i in paths]))))

if not loops:
    print(f"No feedback loops regulate mTOR based on {inputs[1]}ing {inputs[0]}.")
elif len(loops) == 1: 
    print(f"\n Feedback loops acting on mTOR, when {inputs[0]} is {inputs[1]}ed: \n- {loops[0][1]} : {loops[0][0]}")
else:
    print(f"\n Feedback loops acting on mTOR, when {inputs[0]} is {inputs[1]}ed: \n- " + '\n- '.join(map(str, ([f"{i[0]}: {i[1]}" for i in loops]))))

Question: signaling, Response: other [6]
Question: chosen_other, Response: Wnt [22]
Question: direction, Response: activate [1]

 Pathways from Wnt to mTOR, when Wnt is inhibited: 
- activate: ['Wnt', 'Frizzled', 'Dvl', 'GSK3B', 'TBC1D7', 'Rheb', 'mTOR']
- activate: ['Wnt', 'Frizzled', 'Dvl', 'GSK3B', 'TSC1/2', 'Rheb', 'mTOR']
- activate: ['Wnt', 'LRP5/6', 'Dvl', 'GSK3B', 'TBC1D7', 'Rheb', 'mTOR']
- activate: ['Wnt', 'LRP5/6', 'Dvl', 'GSK3B', 'TSC1/2', 'Rheb', 'mTOR']

 Feedback loops acting on mTOR, when Wnt is inhibited: 
- positive_feedback: ['S6K', 'IRS1', 'PI3K', 'AKT2', 'mTOR']
- positive_feedback: ['S6K', 'IRS1', 'PI3K', 'PIP3', 'PDK1', 'Akt', 'IKKa', 'mTOR']
- positive_feedback: ['S6K', 'IRS1', 'PI3K', 'PIP3', 'PDK1', 'Akt', 'PRAS40', 'Raptor', 'mTOR']
- positive_feedback: ['S6K', 'IRS1', 'PI3K', 'PIP3', 'PDK1', 'Akt', 'PRAS40', 'mTOR']
- negative_feedback: ['S6K', 'IRS1', 'PI3K', 'PIP3', 'PDK1', 'Akt', 'TBC1D7', 'Rheb', 'mTOR']
